In [1]:
# Import necessary libraries
# sentence-transformers for embeddings, chromadb for vector store, langchain_community for LLM pipeline
from sentence_transformers import SentenceTransformer
import chromadb
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline  # Changed to Seq2SeqLM for T5
import pandas as pd

# Note: Ensure requirements.txt includes langchain-community, transformers, torch, etc.

In [2]:
# Load the embedding model from Task 2
# all-MiniLM-L6-v2 chosen for efficiency (384 dims, 0.5s/batch on 4-core CPU)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
# Initialize ChromaDB client and load the collection
# Path points to the 3GB vector store from Task 2
client = chromadb.PersistentClient(path="../data/embeddings/")
collection = client.get_collection(name="complaint_embeddings")

In [4]:
# Load LLM (attempt Mistral-7B with authentication, fallback to flan-t5-base, then OpenAI)
# Mistral-7B requires Hugging Face access; flan-t5-base uses Seq2Seq; OpenAI needs API key
from huggingface_hub import login
try:
    # Attempt to log in with your token (replace with your actual token)
    login(token="place your hf token here")  # Request access at https://huggingface.co/mistralai/Mixtral-7B-Instruct-v0.2
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)  # CausalLM for Mistral
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
    llm = HuggingFacePipeline(pipeline=pipe)
except Exception as e1:
    print(f"Error loading Mistral-7B: {e1}. Trying flan-t5-large.")
    try:
        model_name = "google/flan-t5-large"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Corrected for T5
        pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)  # Changed pipeline type
        llm = HuggingFacePipeline(pipeline=pipe)
    except Exception as e2:
        print(f"Error loading flan-t5-base: {e2}. Using OpenAI API as final fallback.")
        from langchain_openai import OpenAI
        llm = OpenAI(api_key="your-api-key-here")  # Replace with your OpenAI API key

Error loading Mistral-7B: Invalid user token.. Trying flan-t5-large.


Device set to use cpu
C:\Users\hp\AppData\Local\Temp\ipykernel_31676\3160485573.py:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [6]:
# Define retriever function
# Retrieves top-k chunks based on query embedding, with optional product filter
def retrieve_chunks(query: str, product_filter: str = None, k: int = 3) -> list:
    query_embedding = embedding_model.encode([query])
    product_map = {
        "credit cards": "Credit Cards",
        "bnpl": "Buy Now, Pay Later (BNPL)",
        "money transfers": "Money Transfers",
        "personal loans": "Personal Loans",
        "savings accounts": "Savings Accounts"
    }
    if product_filter:
        filter_product = product_map.get(product_filter.lower(), product_filter)
        print(f"Filtering for product: {filter_product}")
        results = collection.query(
            query_embeddings=query_embedding,
            where={"mapped_product": filter_product},
            n_results=k
        )
        print(f"Retrieved documents for {filter_product}: {results['documents'][0]}")
    else:
        print("No product filter applied")
        results = collection.query(query_embeddings=query_embedding, n_results=k)
    return results['documents'][0]

In [7]:
# Define prompt template
# Guides LLM to act as a financial analyst, using only provided context
# Define prompt template (refined for clarity)
prompt_template = """
You are a financial analyst assistant for CreditTrust. Your task is to analyze customer complaints based on the provided context.
You MUST use ALL relevant details from the context to summarize the main issues or reasons for complaints in a clear, concise paragraph. Include at least one specific example (e.g., late fees, fraud disputes, payment timing issues) directly from the context. If the context is empty or completely irrelevant to the question, and only then, state: 'I don’t have enough information to provide a detailed answer.'
Context: {context}
Question: {question}
Answer:
"""

In [8]:
# Define generator function
# Combines prompt, question, and chunks to generate response
def generate_response(question: str, product_filter: str = None) -> str:
    if llm is None:
        return "Model loading failed. No free fallback available."
    chunks = retrieve_chunks(question, product_filter)
    context = "\n".join(chunks)
    prompt = prompt_template.format(context=context, question=question)
    response = llm(prompt)
    return response

In [9]:
# Test questions for evaluation
# Covers all five product categories
test_questions = [
    "Why are people unhappy with Credit Cards?",
    "What are the main BNPL issues?",
    "Are there fraud signals in Money Transfers?",
    "What problems are reported with Personal Loans?",
    "Why do customers complain about Savings Accounts?",
    "How often do Credit Card complaints occur monthly?",
    "What causes delays in BNPL payments?"
]

In [10]:
# Debug: Validate vector store content and check for duplicates
total_docs = collection.count()
print(f"Vector store contains {total_docs} documents. Validating unique entries...")
ids = collection.get()['ids']
unique_ids = len(set(ids))
print(f"Estimated unique documents: {unique_ids} (no duplicates detected)")

Vector store contains 770308 documents. Validating unique entries...
Estimated unique documents: 770308 (no duplicates detected)


In [11]:
# Generate responses and collect evaluation data
# Matches questions to products for filtering
evaluation_data = []
for question in test_questions:
    product_filter = next((p for p in ["Credit Cards", "BNPL", "Money Transfers", "Personal Loans", "Savings Accounts"] 
                         if p.lower() in question.lower()), None)
    response = generate_response(question, product_filter=product_filter)
    chunks = retrieve_chunks(question, product_filter=product_filter)
    eval_entry = {
        "Question": question,
        "Generated Answer": response[:500] + "..." if len(response) > 500 else response,
        "Retrieved Sources": chunks[:2],
        "Quality Score": None,
        "Comments": "Pending review"
    }
    evaluation_data.append(eval_entry)

Filtering for product: Credit Cards
Retrieved documents for Credit Cards: ['case description complaint made bank one subsidiaries elan financial services ultimately responsible well hidden policies created bank responsible party complaint solicitation received mail signed ent credit card offered elan financial servies division bank part reason getting card generous offered certain categories home utilities based solicitation decided get card selected categories incurred convenience fee order pay home utility colorado transaction posted called told obvious mistake would fixed fixed', 'suspected based two separate notifications believed sent counterfeit merchandise indeed item counterfeit asked notify local law enforcement give suspected counterfeit merchandise postal inspector believed knowingly shipping counterfeit items crime specifically code since per believed knowingly shipped counterfeit items per wanted nothing items since may counterfeit told synchrony bank would get back opened

C:\Users\hp\AppData\Local\Temp\ipykernel_31676\1480524777.py:9: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


Filtering for product: Credit Cards
Retrieved documents for Credit Cards: ['case description complaint made bank one subsidiaries elan financial services ultimately responsible well hidden policies created bank responsible party complaint solicitation received mail signed ent credit card offered elan financial servies division bank part reason getting card generous offered certain categories home utilities based solicitation decided get card selected categories incurred convenience fee order pay home utility colorado transaction posted called told obvious mistake would fixed fixed', 'suspected based two separate notifications believed sent counterfeit merchandise indeed item counterfeit asked notify local law enforcement give suspected counterfeit merchandise postal inspector believed knowingly shipping counterfeit items crime specifically code since per believed knowingly shipped counterfeit items per wanted nothing items since may counterfeit told synchrony bank would get back opened

In [13]:
# Create evaluation table
eval_df = pd.DataFrame(evaluation_data)
print("\nEvaluation Table:")
print(eval_df.to_markdown(index=False))

# Save evaluation data for report integration
eval_df.to_csv("../data/evaluation_results.csv", index=False)


Evaluation Table:
| Question                                           | Generated Answer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Retrieved Sources                                                                                                                                                                                                                                                                                                                                                                                                                   